In [59]:
!pip install pymongo
import pymongo

In [60]:
!pip install keras
!pip install tensorflow

In [61]:
from keras.models import model_from_json

In [62]:
def load_saved_model_from_db(model_name, client, db, dbconnection):
    json_data = {}
    myclient = pymongo.MongoClient(client)
    
    mydb = myclient[db]
    mycon = mydb[dbconnection]
    data = mycon.find({'created_time':1670573632.139961})
    
    for i in data:
        json_data = i
        
    keras_model = model_from_json(json_data[model_name])
    
    return keras_model

In [63]:
model  = load_saved_model_from_db(model_name = 'modelKafka', client ='mongodb://Container_IP/', db = 'bigData', dbconnection = 'MLModels')

In [64]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               2432      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 1

In [78]:
import tensorflow as tf
opt = "adam"
Loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
evaluations = ['accuracy']



In [79]:
model.compile(optimizer= opt, loss= Loss, metrics= evaluations) #just model.compile() is enough for testing

In [80]:
import pandas as pd

In [81]:
accuracy = []

In [82]:
for df in pd.read_csv("/readTest/SUSY.CSV", chunksize = 100000):
    X_test = df.iloc[:, 1:].to_numpy()
    Y_test = df.iloc[:, 0].to_numpy()
    score = model.evaluate(X_test, Y_test)
    accuracy.append(score[1])

/opt/conda/lib/python3.10/site-packages/keras/backend.py:5676: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


  83/3125 [..............................] - ETA: 3s - loss: 0.6997 - accuracy: 0.4661

/opt/conda/lib/python3.10/site-packages/keras/backend.py:5676: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


3125/3125 [==============================] - 4s 1ms/step - loss: 0.7005 - accuracy: 0.4603


In [83]:
len(accuracy) # 5 Million Records with 100K chunksize therefore 50 records

50

In [87]:
final_test_accuracy = sum(accuracy) / 50

In [89]:
print("Final test accuracy of the entire dataset: " + str(final_test_accuracy))

Final test accuracy of the entire dataset: 0.45899408996105195
